In [42]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import dlib

In [43]:
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("./models/shape_predictor_68_face_landmarks.dat")

In [44]:
cap = cv2.VideoCapture(0)
ALERT_DURATION = 15
closed_eyes_counter = 0
threshold = 120  
alert_threshold = 15

In [45]:
model = load_model('drowsiness_model.h5')

In [46]:
def extract_eye_roi(gray, landmarks, eye_points):
    eye = np.array([(landmarks.part(p).x, landmarks.part(p).y) for p in eye_points])
    x, y, w, h = cv2.boundingRect(eye)
    roi = gray[y:y+h, x:x+w]
    return roi

In [47]:
from playsound import playsound
import threading

def play_alert_sound():
    playsound('./alert.mp3')  # Put this file in the same folder or use full path


In [48]:
LEFT_EYE_POINTS = list(range(36, 42))
RIGHT_EYE_POINTS = list(range(42, 48))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)

    eye_open = False
    focus_status = "Not Focused"

    for face in faces:
        landmarks = landmark_predictor(gray, face)

        for eye_points in [LEFT_EYE_POINTS, RIGHT_EYE_POINTS]:
            eye_roi = extract_eye_roi(gray, landmarks, eye_points)

            if eye_roi.size == 0 or eye_roi.shape[0] < 10 or eye_roi.shape[1] < 10:
                continue

            eye_roi_resized = cv2.resize(eye_roi, (48, 48))
            eye_roi_resized = eye_roi_resized.astype("float32") / 255.0
            eye_roi_resized = np.expand_dims(eye_roi_resized, axis=-1)
            eye_roi_resized = np.expand_dims(eye_roi_resized, axis=0)

            prediction = model.predict(eye_roi_resized, verbose=0)[0]
            open_prob = prediction[1]

            if open_prob > 0.6:
                eye_open = True
                break  

        cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()),
                      (0, 255, 0) if eye_open else (0, 0, 255), 2)

    if eye_open:
        focus_status = "Focused"
        closed_eyes_counter = 0
    else:
        closed_eyes_counter += 1

    cv2.putText(frame, f" {focus_status}", (20, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0) if focus_status == "Focused" else (0, 0, 255), 2)

    if closed_eyes_counter >= ALERT_DURATION:
        cv2.putText(frame, "DROWSINESS ALERT!", (20, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        threading.Thread(target=play_alert_sound).start()

    cv2.imshow("Drowsiness Detector (Dlib)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ERROR:playsound:
    Error 265 for command:
        open ./alert.mp3
    The device name is already being used as an alias by this application.  Use a unique alias.
Exception in thread Thread-1706:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\User\Desktop\BSCS\6th Semester\Computer Vision\CV project final\venv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_16012\306507756.py", line 5, in play_alert_sound
  File "c:\Users\User\Desktop\BSCS\6th Semester\Computer Vision\CV project final\venv\lib\site-packages\playsound.py", line 72, in